### Send an email with a csv attachment via API, using Microsoft Graph Registered app

1. **Register an App in Azure AD:**
   - Go to the **Azure Portal** and navigate to **Azure Active Directory**.
   - Select **App registrations** > **New registration**.
   - Provide a **name** for your app, choose the appropriate **supported account types**, and set the **redirect URI** if needed.
   - Click **Register**.

2. **Configure API Permissions:**
   - In your registered app, go to **API permissions**.
   - Click **Add a permission** > **Microsoft Graph** > **Delegated permissions**.
   - Select the following permissions:
     - `Mail.Send`
   - Click **Add permissions**.
   - Grant admin consent.

3. **Create a Client Secret:**
   - In the app's **Certificates & secrets** section, click **New client secret**.
   - Provide a description and expiration period, then click **Add**.
   - Copy the **Client Secret** value, as you will need it later.

4. **Obtain OAuth2 Token:**
   - Use the **Client ID**, **Client Secret**, and **Tenant ID** to request an OAuth2 token.
   - Send a POST request to `https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token` with the following parameters:
     - `client_id`: Your app's Client ID
     - `scope`: `https://graph.microsoft.com/.default`
     - `client_secret`: Your app's Client Secret
     - `grant_type`: `client_credentials`


In [11]:
import requests
import json
import base64
from datetime import datetime
import glob
import os

StatementMeta(, d958ceb5-14e3-4627-b578-4bf0170a0672, 16, Finished, Available, Finished)

In [12]:
%run Bronze | Functions

StatementMeta(, d958ceb5-14e3-4627-b578-4bf0170a0672, 20, Finished, Available, Finished)

In [13]:
current_date = datetime.now(timezone).date()

StatementMeta(, d958ceb5-14e3-4627-b578-4bf0170a0672, 21, Finished, Available, Finished)

In [14]:
# Get client credentials
client_id = mssparkutils.credentials.getSecret('KEYVAULTLINK','APPCLIENTID')
tenant_id = mssparkutils.credentials.getSecret('KEYVAULTLINK','TENANTID')
client_secret = mssparkutils.credentials.getSecret('KEYVAULTLINK','APPCLIENTSECRET')

StatementMeta(, d958ceb5-14e3-4627-b578-4bf0170a0672, 22, Finished, Available, Finished)

In [15]:
user_email = "user@email.com"

# Function to request the bearer token using Client Credentials Flow
def request_bearer_token(tenant_id, client_id, client_secret):
    token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    
    # Request body
    body = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": "https://graph.microsoft.com/.default"
    }
    
    # Make the POST request
    response = requests.post(token_url, data=body)
    
    if response.status_code == 200:
        token_response = response.json()
        return token_response["access_token"]
    else:
        print(f"Failed to obtain bearer token. Status Code: {response.status_code}, Response: {response.text}")
        return None

# Function to send an email with an attachment via Microsoft Graph API
def send_email_with_attachment(access_token, user_email, to_email, subject, body, attachment_path):
    graph_api_url = f"https://graph.microsoft.com/v1.0/users/{user_email}/sendMail"
    
    # Read attachment file
    with open(attachment_path, "rb") as file:
        attachment_content = file.read()
    
    # Convert file to Base64
    encoded_attachment = base64.b64encode(attachment_content).decode('utf-8')
    
    # Email payload with attachment
    email_payload = {
        "message": {
            "subject": subject,
            "body": {
                "contentType": "Text",
                "content": body
            },
            "toRecipients": [
                {
                    "emailAddress": {
                        "address": to_email
                    }
                }
            ],
            "attachments": [
                {
                    "@odata.type": "#microsoft.graph.fileAttachment",
                    "name": os.path.basename(attachment_path),
                    "contentType": "text/csv",
                    "contentBytes": encoded_attachment
                }
            ]
        },
        "saveToSentItems": "true"
    }
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    # Make the POST request to send the email
    response = requests.post(graph_api_url, headers=headers, json=email_payload)
    
    if response.status_code == 202:
        print("Email sent successfully.")
    else:
        print(f"Failed to send email. Status Code: {response.status_code}, Response: {response.text}")

StatementMeta(, d958ceb5-14e3-4627-b578-4bf0170a0672, 23, Finished, Available, Finished)

In [16]:
df = spark.sql("SELECT * FROM lakehouse.etl_table")

# Filter example, the data you want to sent as csv attachment
df_failure = df.filter((df.run_status == "failure") & (df.date == current_date))

if df_failure != None:

    current_date = datetime.now(timezone).strftime("%Y%m%d")
    csv_path = f"Files/ETL KPI failure records/{current_date}/"

    df_failure.coalesce(1).write.mode("overwrite").option("header", "true").format("csv").save(csv_path)

    csv_file_path = f"/lakehouse/default/Files/ETL failure records/{current_date}/*.csv"
    csv_files = glob.glob(csv_file_path)

    csv_file = csv_files[0]

    # Step 1: Request token
    access_token = request_bearer_token(tenant_id, client_id, client_secret)
        
    # Step 2: Send an email with attachment if the token was successfully obtained
    if access_token:
        send_email_with_attachment(
            access_token = access_token,
            user_email = user_email,
            to_email = user_email,
            subject = "ETL KPI Failure Records",
            body = "Please find attached the ETL KPI failure records.",
            attachment_path = csv_file
        )
    else:
        print("Missing token.")

else:
    print("No CSV files found in the directory.")

StatementMeta(, d958ceb5-14e3-4627-b578-4bf0170a0672, 24, Finished, Available, Finished)

Email sent successfully.
